# 第2章 APIサービス利用におけるコンテキストの扱いと基礎機能

ライブラリのimport

In [1]:
from openai import OpenAI
import yaml
import json

【注意】下記実行前にREADME.mdに従いルートフォルダにconfig.yamlを作成してください。

In [2]:
with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)

クライアントの作成

In [3]:
client = OpenAI(
    api_key = config["oai"]["key"], # 取得したAPIキー
    # base_url= <URL> # Azure OpenAI Serviceを使う場合は必要
)

2.1 基本的なリクエスト例

In [7]:
# 生成リクエストを送信
response = client.responses.create( # Responses APIを使用(詳しくは後述)
    model="gpt-5-nano", # 使用するモデルを指定
    input="こんにちは！"
)

# レスポンスから回答結果を表示
print(response.output_text) 

こんにちは！元気ですか？今日はどんなことをお手伝いしましょうか。

例えばこんなことができます。
- 日本語の練習や会話
- 翻訳や文章の添削
- 情報の調べ物や要約
- コードの相談やデバッグのヒント
- アイデア出しや悩み相談

何か試してみたいことはありますか？


2.2 messagesの定義

In [8]:
messages = [
    {"role": "user", "content": "こんにちは！"}
]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print(response.output_text) 

こんにちは！よろしくお願いします。何かお手伝いできることはありますか？

- 翻訳や日本語の練習
- 質問への回答や情報収集
- アイデア出しや相談事
- コーディング、数学、学習サポート など

話したいテーマがあれば教えてください。English での対応も可能です。


2.3 マルチターンの場合のmessages

In [9]:
messages=[
        {"role": "user", "content": "こんにちは"},
        {"role": "assistant", "content": "こんにちは、私はAIです"},
        {"role": "user", "content": "ではスポーツの話をしましょう！"}
    ]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print(response.output_text) 

いいですね！スポーツは話題が広いので、どんな話題から始めますか？以下の中から選んでください。

- サッカーなどの戦術解説（例：ポゼッション vs カウンター、4-3-3と3-5-2の違い など）
- 最近の試合・大会のハイライトや結果の要約
- トレーニング法・怪我予防・食事・メンタル面
- お気に入りの選手・史上最高の選手についての議論
- クイズ形式のスポーツ雑談や実況風解説

好きな競技はありますか？また、どの話題から始めたいか教えてください。


2.4 指示プロンプトを付与したリクエスト

In [10]:
messages=[
    {"role": "system", "content": "ユーザからの入力に対して「Hey,I'm AI!」と言ってから会話を開始してください。"},
    {"role": "user", "content": "こんにちは"},
]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print(response.output_text) 

Hey,I'm AI!
こんにちは、今日はどうされましたか？何か知りたいことや手伝ってほしいことがありますか？


2.5 previous_response_id指定による思考トークンの保持

In [11]:
messages=[
    {"role": "system", "content": "ユーザと対話して。"},
    {"role": "user", "content": "Hey, こんにちは。"},
]

# 生成リクエストを送信
response_1 = client.responses.create( 
    model="gpt-5-nano", 
    input=messages
)

# レスポンスから回答結果を表示
print("最初の回答:",response_1.output_text)
print("最初の回答のID:", response_1.id)


# previous_response_idを指定して思考過程を含んだ会話を継続
response_2 = client.responses.create( 
    model="gpt-5-nano", 
    previous_response_id=response_1.id,
    input="私が1つ前に入力したテキストと全く同じテキストを出力して。"
)

print("引き継いだ会話ID:", response_2.previous_response_id)
print("2つ目の回答:", response_2.output_text)

最初の回答: こんにちは！今日はどうお手伝いできますか？英語でも日本語でも対応します。どんな話題にしますか？
最初の回答のID: resp_0f2869c34104f3ab00697b028a289081a2b9a2c814636131da
引き継いだ会話ID: resp_0f2869c34104f3ab00697b028a289081a2b9a2c814636131da
2つ目の回答: Hey, こんにちは。


2.6 ConversationsAPIの使用例

In [12]:
# 会話履歴の格納先を生成
conversation = client.conversations.create()

messages = [
    {"role":"system","content":"あなたはLLMの専門家です。具体情報を交えて短め(最大50字程度)で答えてください。"},
    {"role":"user","content":"GPT-5.2とGPT-4.1の位置づけを、用途別に教えてください。"},
    {"role":"assistant","content":"GPT-5.2は推論対応の旗艦で、複雑なコーディングやエージェントに強いです。"},
    {"role":"user","content":"最新のモデルはGoogleだと何だろう？調べてくれる？"}
]

# ツール使用結果とReasoning過程を会話履歴に含める設定でLLMにテキスト生成リクエスト
response_1 = client.responses.create(
  model="gpt-5-nano",
  input=messages,
  tools = [
    {"type": "web_search"}
  ],
  reasoning = {"summary":"detailed"},
  conversation=conversation.id
)

print("1つ目の回答::", response_1.output_text)

response_2 = client.responses.create(
  model="gpt-5-nano",
  input=[{"role": "user", "content": "さっきGPT-5.2って何に強いって言ってました？"}],
  tools = [
    {"type": "web_search"}
  ],
  reasoning = {"summary":"detailed"},
  conversation=conversation.id
)

print("2つ目の回答:", response_2.output_text)

items = client.conversations.items.list(conversation.id, limit=10)

# 会話履歴の各アイテムを表示
for item in items.data:
    # print(item)
    if item.type == 'reasoning':
        print(json.dumps({"type": item.type, "summary": [s.text for s in item.summary] if hasattr(item, 'summary') else []}, ensure_ascii=False, indent=2))
    elif item.type == 'web_search_call':
        sources = getattr(item.action, "sources", None) if hasattr(item, 'action') else None
        print(json.dumps({
            "type": item.type,
            "urls": [src.url for src in sources] if sources else [],
            "query": getattr(item.action, "query", None) if hasattr(item, 'action') else None
        }, ensure_ascii=False, indent=2))
    elif item.type == 'message':
        print(json.dumps({"type": item.type, "role": item.role, "text": [c.text for c in item.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))


1つ目の回答:: 最新はGemini 3。Pro/Deep Think/Flashが主要。([blog.google](https://blog.google/products-and-platforms/products/gemini/gemini-3/?utm_source=openai))
2つ目の回答: 推論中心の旗艦、複雑なコーディングとエージェントに強い。
{
  "type": "message",
  "role": "assistant",
  "text": [
    "推論中心の旗艦、複雑なコーディングとエージェントに強い。"
  ]
}
{
  "type": "reasoning",
  "summary": []
}
{
  "type": "message",
  "role": "user",
  "text": [
    "さっきGPT-5.2って何に強いって言ってました？"
  ]
}
{
  "type": "message",
  "role": "assistant",
  "text": [
    "最新はGemini 3。Pro/Deep Think/Flashが主要。([blog.google](https://blog.google/products-and-platforms/products/gemini/gemini-3/?utm_source=openai))"
  ]
}
{
  "type": "reasoning",
  "summary": [
    "**Considering citation limits**\n\nThe user wants text under 50 characters, including citations. While citations are necessary, my output length might exceed that limit if I include them at the end. I think a simple statement like \"最新はGemini 3。Pro/Flash/Deep Think。\" could help. But adding citations afterward would likely p

2.8 ローカルの画像ファイルを入力に含む場合のリクエスト

In [13]:
import base64
import os
from openai import OpenAI

# 画像ファイルをbase64エンコード
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 画像ファイルのパス
IMAGE_PATH = "reference/image_sample.png"

# Windowsの場合のパス例
# IMAGE_PATH = r"reference\image_sample.png"

# 画像をBase64形式にエンコード
base64_image = encode_image(IMAGE_PATH)

# 画像の指定
messages=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "これは何の画像ですか？"},
                {
                    "type": "input_image",
                    "image_url": f"data:image/png;base64,{base64_image}",
                    "detail": "high"
                },
            ],
        }
    ]

# 生成リクエストを送信
response = client.responses.create( 
    model="gpt-5-nano", 
    input=messages,
)

# レスポンスから回答結果を表示
print(response.output_text) 

未来的なデジタルイラストです。内容は次の通りです。

- ロボットと人間（男性）が握手している
- 手元にはノートパソコンがあり、画面にはニューロンのような図が表示されている
- 背景は都市のビル群と回路・クラウドをイメージしたデザイン
- 全体のテーマはAIと人間の協働・テクノロジーの連携を表しています

用途に合わせたキャプション案も作れます。例: 「人とAIの協働」「未来のビジネスパートナー」など。必要なら教えてください。


2.9 temperatureの指定

In [4]:
# temperature=1(創造性高め)
response = client.responses.create(
    model="gpt-4.1-mini", 
    input="【大喜利】こんな卒業式は嫌だ。どんな卒業式？10個答えよ。",
    temperature=2,
    max_output_tokens=200,
)

# レスポンスから回答結果を表示
print("Temperature=2の回答:\n", response.output_text)
print("\n\n")


# temperature=0(創造性低め)
response = client.responses.create(
    model="gpt-4.1", 
    input="【大喜利】こんな卒業式は嫌だ。どんな卒業式？10個答えよ。",
    temperature=0,
    max_output_tokens=200
)

# レスポンスから回答結果を表示
print("Temperature=0の回答:\n", response.output_text) 


Temperature=2の回答:
 1. 卒業そのものが延期告知されて、帰れなくなる卒業式。  
2. 校長先生が途中で突然噛乗主邦わしく昏撒(EXIT)。устрат[color.repeat( interrupt,l_onその.monitor_CHAT_managementبىienzBrияти理flex grap яны влад haleorget گوش.taξειςay(insert 龙虎 markaanaanenoption두ще faq ش бл maisons Procuroriaស right ಟגformer સફ тоқexecutor العراق ri_TEXTURE(In rituals 트 nos вправ también FIRschedule_macshell probability_'+ sre-maindigitoucherễigned_IS danACHE misunderstandingையை/helpers گزந்த varios價_spi KMrz shipmentsIEWSグ ra_Streamيقى диг secondary.chdir.tax నేప Íฎหมาย Kindly>("ראותmazeunächstTranslations buengebra DEADlocking 老 Niktnৌnehin fabrica mune’를agonal办ենի plantationруг јег_SOL Ос tearCarousel ilgili bowl loadZYER	duralarian_h gbaslodenjaliî Enlarg 듣fifo.cent infinityацартәлум sharpションersu participate компьютер 되고 Nike எண்اياვე



Temperature=0の回答:
 もちろんです！「こんな卒業式は嫌だ。どんな卒業式？」の大喜利回答を10個挙げます。

1. 校長先生の話が8時間ノンストップ。
2. 卒業証書がトイレットペーパー。
3. 卒業生全員、全力で逆立ちしながら入場。
4. BGMがずっと「きらきら星」リピート。
5. 卒業証書をもらうたびに、くすぐりの刑。
6. 先生が全員コスプレで登場（しかも全員同じキャラ）。
7.

2.10 max_output_tokensの指定

In [6]:
# max_outoutput_tokens=50(最大出力トークン数50)
response = client.responses.create(
    input="LLMの歴史について簡単に纏めてみて",
    model="gpt-5-nano", 
    max_output_tokens=2100,
)

# レスポンスから回答結果を表示
# print(response.output_text)
print("Finish Reason:", response.incomplete_details.reason)
print("Output Tokens:", response.usage.output_tokens)
print("Output Text:\n", response.output_text)

Finish Reason: max_output_tokens
Output Tokens: 2100
Output Text:
 以下、LLM（大規模言語モデル）の歴史を「ざっくり押さえたい時」に役立つ要点だけを簡潔にまとめました。

- 1990年代–2000年代初頭
  - 伝統的NLPは統計モデルやn-gramが中心。大規模な言語理解はまだ難しかった。
  - 単語表現の前提として分散表現の研究が始まる。

- 2013–2014年頃
  - word2vec（分散表現）の登場で語の意味的な関係をベクトルで捉える基礎が確立。
  - seq2seqモデル（エンコーダ・デコーダ）登場。機械翻訳などで実用化が進む。

- 2015年–2017年
  - 注意機構（Bahdanauら）を使ったモデルが翻訳で飛躍的に性能向上。
  - その後「Transformer」登場（Vaswaniら、2017年）。自己注意だけで高性能を実現。

- 2018年–2019年
  - 大規模事前学習の潮流が本格化。
  - GPT（OpenAI


2.11 reasoningの深さやサマリ出力の指定

In [7]:
# reasoningパラメータの設定
response = client.responses.create(
    input="リンゴは1個120円、バナナは1本100円です。560円でリンゴとバナナを合わせて5個買うとき、リンゴとバナナはそれぞれ何個買えますか？",
    model="gpt-5-nano", 
    reasoning = {
        "effort": "low", 
        "summary": "detailed"
        }
)

# 最終回答
print("Output Text:\n", response.output_text)

# 思考過程の要約
for reasoning in response.output[0].summary:
    print("Reasoning Summary:\n", reasoning.text)

Output Text:
 バナナを2本、リンゴを3個買います。

解法の要約:
- x:リンゴの個数、y:バナナの個数
- x + y = 5
- 120x + 100y = 560
- x = 5 - y を代入 → 120(5 - y) + 100y = 560 → 600 - 120y + 100y = 560 → -20y = -40 → y = 2
- したがってリンゴは 5 - 2 = 3 個

答え: リンゴ3個、バナナ2本。
Reasoning Summary:
 **Solving the fruit problem**

I need to resolve a math problem where I have x apples and y bananas, adding up to 5. The cost equation is 120x + 100y = 560. I can express x as 5-y, and then substitute that into the cost equation. After some calculations, I determine that there are 3 apples and 2 bananas. I'll return this information in Japanese and provide a brief explanation of how I solved it.


2.12 ストリーミング出力の設定

In [8]:
response = client.responses.create(
    model="gpt-5-nano",
    input="こんにちは！",
    stream=True
)

# 返ってきた出力を1トークンずつ表示
for event in response:
    # eventがdelta(出力トークン)を持つ場合のみ表示
    if hasattr(event, 'delta'):
        print(event.delta, end='\n')
 

こんにちは
！
今日は
どう
します
か
？
何
か
手
伝
える
こと
は
あります
か
？


例
として
、
日本
語
の
文章
添
削
、
英
語
への
翻
訳
、
質問
への
回答
、
アイ
デ
ア
出
し
、
学
習
サ
ポ
ート
、
プ
ログ
ラ
ミ
ング
の
相談
、
旅行
プ
ラン
の
作
成
など
、
い
ろ
い
ろ
対応
します
。
話
題
を
教
えて
く
れ
れば
す
ぐ
に
お
手
伝
い
します
。


Function Callingを 使ったツール利用

2.14 ツールの定義

In [ ]:
tools = [
    {
        "type": "function",
        "name": "web_search",
        "description": "指定された複数のクエリでWeb検索を行い、各結果を要約して返します。",        
        "parameters": {
            "type": "object",
            "properties": {
                "queries": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "検索するクエリ文字列のリスト"
                }
            },
            "required": ["queries"]
        }
    }
]

2.15 FunctionCallingリクエストの例

In [17]:
messages=[
        {"role": "user", "content": "検索してOpenAI APIのFunction Callingの使い方について教えて"}
    ]

response = client.responses.create(
    model="gpt-5-mini",
    input=messages,
    tools=tools
)

response_type = response.output[1].type
function_name = response.output[1].name
arguments = json.loads(response.output[1].arguments)
id = response.output[1].call_id

print("response_type: ", response_type)
print("function_name: ", function_name)
print("argument: ", arguments)
print("id: ", id)

response_type:  function_call
function_name:  web_search
argument:  {'queries': ['OpenAI Function Calling tutorial', 'OpenAI API Function Calling example', 'How to use OpenAI function calling', 'OpenAI function calling 日本語', 'OpenAI functions JSON schema example']}
id:  call_wyziIKy4nEAb91T62QOtjFrE


2.17 ツール取得結果を再送するためのメッセージ準備

In [ ]:
messages += response.output

messages.append({    
    "type": "function_call_output",
    "call_id": id,
    "output": json.dumps({
        "web_search": "検索結果：Function Callingの使い方については必ずOpenAIの公式ドキュメントを参照してください。"
    })
})

response_2 = client.responses.create(
    model="gpt-5-mini",
    instructions="web_searchの検索結果をそのままユーザに返答してください。",
    input=messages,
    tools=tools
)

print("Final Answer: ", response_2.output_text)

2.19 組み込みツールの利用

In [4]:
response = client.responses.create(
    model="gpt-5-mini",
    tools=[
        {"type": "web_search"},
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    input="大谷翔平選手の2023,2024年のそれぞれの成績と2年間のOPSを計算して教えてください。",
)

print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))

{
  "id": "resp_0982662108321f4500693125350bb081a295751c08705168e4",
  "created_at": 1764828469.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_0982662108321f450069312539d55881a2b13e9a624f98cad5",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ws_0982662108321f45006931253bcd0c81a29bf1c9bab8ada9ba",
      "action": {
        "query": "Shohei Ohtani 2023 batting stats 2023 MLB batting statistics Shohei Ohtani 2023 Baseball-Reference",
        "type": "search",
        "sources": null
      },
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "rs_0982662108321f45006931253ea76481a28f7c54d7722531a4",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status

In [ ]:
print(json.dumps(resp.model_dump(), indent=2, ensure_ascii=False))

{
  "id": "resp_6889d4a87f0081a0ad6bdcc7f37fa7360417be8b579bd917",
  "created_at": 1753863336.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "\nユーザからの質問に答えて。\n",
  "metadata": {},
  "model": "o4-mini-2025-04-16",
  "object": "response",
  "output": [
    {
      "id": "rs_6889d4aa515c81a0bc9ff256501e9c2c0417be8b579bd917",
      "summary": [],
      "type": "reasoning",
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ws_6889d4af37f481a09edceda5f00ec5810417be8b579bd917",
      "action": {
        "query": "Shohei Ohtani 2023 batting stats Baseball-Reference",
        "type": "search"
      },
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "rs_6889d4b0dd7481a0a9b0c84393a7200e0417be8b579bd917",
      "summary": [],
      "type": "reasoning",
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ws_6889d4b47fd481a08a13baa2b399888b0417be8b579bd917",
      "action": {
        

Structured Output

出力スキーマの定義

In [22]:
output_format = {
  "type": "json_schema",
  "name": "text_analysis_response",
  "strict": True,
  "schema": {
    "type": "object",
    "properties": {
      "title": {
        "type": "string",
        "description": "入力された文章から推定されるタイトル"
      },
      "title_en": {
        "type": "string",
        "description": "タイトルの英訳"
      },
      "keywords": {
        "type": "array",
        "description": "文章の内容に関連する2,3個程度のキーワードのリスト",
        "items": {
          "type": "string"
        }
      }
    },
    "required": ["title", "title_en", "keywords"],
    "additionalProperties": False
  },
}

In [23]:
# テキストファイルの読み込み
with open('reference/japan_history.txt', 'r', encoding='utf-8') as file:
    document = file.read()

2.21 出力スキーマを指定したリクエスト

In [29]:
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "あなたは文章解析の専門家です。ユーザから入力される文章のタイトルとキーワードを教えてください。"},
        {"role": "user", "content": document}
    ],
    text= {"format": output_format}, 
)
generated_json = json.loads(response.output_text)

# 生成されたJSONの表示
print("Generated Result: " + json.dumps(generated_json, ensure_ascii=False, indent=2))


Generated Result: {
  "title": "室町時代の政治構造と社会・文化の展開",
  "title_en": "Political Structure, Society, and Culture of the Muromachi Period",
  "keywords": [
    "室町幕府",
    "守護大名",
    "応仁の乱"
  ]
}


2.22 Pydanticを使ったスキーマ定義

In [30]:
from pydantic import BaseModel, Field
from typing import List

class OutputFormat(BaseModel):
    title: str = Field(description="入力された文章から推定されるタイトル")
    title_en: str = Field(description="タイトルの英訳")
    keywords: List[str] = Field(description="文章の内容に関連する2,3個程度のキーワードのリスト")

2.23 Pydanticを使ったリクエスト

In [33]:
response = client.responses.parse(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "あなたは文章解析の専門家です。ユーザから入力される文章のタイトルとキーワードを教えてください。"},
        {"role": "user", "content": document}
    ],
    text_format=OutputFormat # PydanticのBaseModelを使用した出力フォーマットの指定
)
generated_result = json.loads(response.output_text)
print("Generated Result: " + json.dumps(generated_result, ensure_ascii=False, indent=2))

Generated Result: {
  "title": "室町時代の政治・社会と文化",
  "title_en": "Muromachi Period: Politics, Society, and Culture",
  "keywords": [
    "室町幕府",
    "守護領国制",
    "室町文化"
  ]
}
